In [10]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tqdm import tqdm_notebook

In [11]:
# 存放接下來 tensorboard 所有紀錄的路徑
pathLog = 'log/'
os.makedirs(pathLog, exist_ok = True)

In [12]:
# Load MINIST dataset
from tensorflow.examples.tutorials.mnist import input_data

# 可能會出現大量的 Warning ，可以忽視
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)
trainImages, trainLabels = mnist.train.images, mnist.train.labels
testImages, testLabels = mnist.test.images, mnist.test.labels

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# Define the graph

In [14]:
import tensorflow.contrib.slim.nets as slimNet

ImportError: cannot import name 'nets' from 'tensorflow_core.contrib.slim' (/home/hsuchaochun/anaconda3/lib/python3.7/site-packages/tensorflow_core/contrib/slim/__init__.py)

In [ ]:
tf.reset_default_graph()

X_in = tf.placeholder(dtype=tf.float32, shape=(None, 28*28))
y_in = tf.placeholder(dtype=tf.float32, shape=(None, 10))

images = tf.reshape(X_in, shape=[-1, 28, 28, 1])
# 紀錄 3 張影像在 tensorboard 上
tf.summary.image('Input_images', images, max_outputs = 3)

with tf.variable_scope('ResNet50'):
    net, _ = slimNet.resnet_v2.resnet_v2_50(images, global_pool=False, is_training=True)
    gavp = tf.reduce_mean(net, axis=[1,2])
    logit = tf.layers.dense(gavp, 10)
    # 紀錄 gavp, logit 這兩個 tensor 的權重數值分佈
    tf.summary.histogram('Histogram', gavp)
    tf.summary.histogram('Histogram_logit', logit)
    
with tf.name_scope('Loss'):
    loss = tf.losses.softmax_cross_entropy(onehot_labels = y_in, logits = logit)
    # 紀錄 loss
    tf.summary.scalar('Cross_Entropy', loss)

with tf.name_scope('Optimizer'):
    optimizer = tf.train.AdamOptimizer(1e-6).minimize(loss)

result = tf.equal(tf.argmax(logit, 1), tf.argmax(y_in, 1))
with tf.name_scope('Accuracy'):
    accuracy = tf.reduce_mean(tf.cast(result, tf.float32))
    # 紀錄 accuracy
    tf.summary.scalar('Accuracy', accuracy)
    
# 把上面所有的 tf.summary 整合成一個 operation call
opsSummary = tf.summary.merge_all()

# Training

In [5]:
epochs = 20
bs = 100

In [6]:
sess = tf.Session()

#Creat writer to write data needed for tensorboard
writer = tf.summary.FileWriter(pathLog)

#Show graph of neural network in tensorboard
writer.add_graph(sess.graph)

writerTrain = tf.summary.FileWriter(os.path.join(pathLog, 'train'))
writerTest = tf.summary.FileWriter(os.path.join(pathLog, 'test'))

sess.run(tf.global_variables_initializer())

for i in range(epochs):
    # mini-batch training
    for j in tqdm_notebook(range(mnist.train.num_examples // bs)):
        batch = mnist.train.next_batch(bs)
        sess.run(optimizer, feed_dict = {X_in:batch[0], y_in:batch[1]})

    saver = tf.train.Saver()
    saver.save(sess, os.path.join(pathLog, 'model.ckpt'))

    trainAccuracy = sess.run(accuracy, feed_dict = {X_in:trainImages, y_in:trainLabels})
    testAccuracy = sess.run(accuracy, feed_dict = {X_in:testImages, y_in:testLabels})
    
    print('Epoch %2d: acc = %.3f, test_acc = %.3f' % (i+1, trainAccuracy*100., testAccuracy*100.))

    # Calculate desired information to be shown in tensorboard
    summary = sess.run(opsSummary, feed_dict = {X_in:trainImages, y_in:trainLabels})
    writerTrain.add_summary(summary, global_step=i)
    
    summary = sess.run(opsSummary, feed_dict = {X_in:testImages, y_in:testLabels})
    writerTest.add_summary(summary, global_step=i)

/home/hsuchaochun/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()


NameError: name 'optimizer' is not defined

In [ ]:
from tensorflow.contrib.tensorboard.plugins import projector

In [ ]:
def images_to_sprite(data):
    """Creates the sprite image along with any necessary padding

    Args:
      data: NxHxW[x3] tensor containing the images.

    Returns:
      data: Properly shaped HxWx3 image with any necessary padding.
    """
    # shape == 3 表示少一個 channel 維度，是灰階圖，先擴充一個維度之後複製相同的三次變 RGB
    if len(data.shape) == 3:
        data = np.tile(data[...,np.newaxis], (1,1,1,3))
    data = data.astype(np.float32)
    
    # 每一張 image 扣掉每一張的最小值
    _min = np.min(data.reshape((data.shape[0], -1)), axis=1)
    # 把張數維度 transpose 到最後面再減掉各張的最小值，然後再轉回來 
    data = (data.transpose(1,2,3,0) - _min).transpose(3,0,1,2)
    
    # 每一張 image 的除上每一張的最大值
    _max = np.max(data.reshape((data.shape[0], -1)), axis=1)
    data = (data.transpose(1,2,3,0) / _max).transpose(3,0,1,2)
    
    # 把顏色翻轉應該比較好看，預設是有值為白色、背景是黑色
    data = 1 - data

    n = int(np.ceil(np.sqrt(data.shape[0])))
    
    # 接下來要把所有圖拼成 n x n 的一張 image
    padding = ((0, n ** 2 - data.shape[0]), (0, 0), (0, 0)) + ((0, 0),) * (data.ndim - 3)
    data = np.pad(data, padding, mode='constant', constant_values=0)
    
    # Tile the individual thumbnails into an image.
    # 把每一張圖拼起來變成一張縮圖的 image
    data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3) + tuple(range(4, data.ndim + 1)))
    data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
    data = (data * 255).astype(np.uint8)
    return data

In [7]:
# Generate the sprite image
pathSprite = os.path.join(pathLog, 'mnist_10k_sprite.png')
cv2.imwrite(pathSprite, images_to_sprite(testImages.reshape(-1, 28, 28)))
# !cp mnist_10k_sprite.png log/

# Generate the metadata (here is just the labels of figures)
pathMetadata = os.path.join(pathLog, 'metadata.tsv')
with open(pathMetadata, 'w') as fin:
    for i in np.argmax(testLabels, axis=1):
        fin.write('%s\n' % i)

# Calculate the layer for embedding
# 此處拿 ResNet50 的 2048 維輸出做 embedding
embeddingVar = sess.run(gavp, feed_dict = {X_in:testImages})

# It needs to be a tensorflow variable and initialized
embeddingVar = tf.Variable(embeddingVar, name='embedding')
sess.run(embeddingVar.initializer)

saver = tf.train.Saver()
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = embeddingVar.name
embedding.metadata_path = os.path.basename(pathMetadata)
embedding.sprite.image_path = os.path.basename(pathSprite)
embedding.sprite.single_image_dim.extend([28, 28])
projector.visualize_embeddings(writer, config)

path = os.path.join(pathLog, 'embedding.ckpt')
saver.save(sess, path)

writer.close()
sess.close()

NameError: name 'images_to_sprite' is not defined

In [8]:
# Run the tensorboard server
!tensorboard --logdir='log/'

TensorBoard 1.15.0 at http://hsuchaochundeubuntu:6006/ (Press CTRL+C to quit)
^C
